In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data

import torch.optim as optim

import torchvision
from torchvision import datasets as dsets
from torchvision import transforms

from matplotlib.pyplot import imshow
%matplotlib inline

import visdom
import math
import vgg
import numpy as np

In [ ]:
vis = visdom.Visdom()
vis.close(env="main")

In [ ]:
def plt_tracker(loss_plot, loss_value, num):
    vis.line(X=np.stack(np.arange(num, num+1)),
             Y=np.stack([loss_value]),
             win=loss_plot,
             update='append'
             )

In [ ]:
if (torch.cuda.is_available() == 1):
    print("cuda is available")
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
transform = transforms.Compose([
    #transforms.Resize(256),
    transforms.RandomCrop(56),
    transforms.ToTensor(),]
    #transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))]
)

In [ ]:
try:
    train = dsets.ImageFolder(root='./tiny-imagenet-200/train',transform=transform)
    print("Dataset is alreay downlaoded")
except:
    !./download.sh
    train = dsets.ImageFolder(root='./tiny-imagenet-200/train',transform=transform)
    
        
validation = dsets.ImageFolder(root='./tiny-imagenet-200/val',transform=transform)
test = dsets.ImageFolder(root='./tiny-imagenet-200/test',transform=transform)

print(train)
print(validation)
print(test)


check_img=train.__getitem__(0)
ToPIL=transforms.ToPILImage()
check_img = ToPIL(check_img[0])
print(check_img)

imshow(check_img)

In [ ]:
train_loader=data.DataLoader(train, batch_size=256, shuffle=True, num_workers=2)
var_loader=data.DataLoader(validation, batch_size=64, shuffle=False, num_workers=2)
test_loader=data.DataLoader(test, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
conv = vgg.make_layers(vgg.cfg['tiny-imagenet'],True)

In [ ]:
class tiny_VGG(nn.Module):
    def __init__(self,conv_layer,num_of_class=200, init_weights=True):
        super(tiny_VGG, self).__init__()
        self.conv = conv_layer
        self.fc = nn.Sequential(
            nn.Linear(7*7*512, 1024),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(1024, 1024),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(1024, num_of_class))
        
        if init_weights:
            self._initialize_weights()
        
    def forward(self,x):
        x = self.conv(x)
        #print(x.shape)
        x=x.view(x.shape[0], -1)
        
        x=self.fc(x)
        return x
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [ ]:
CNN = tiny_VGG(conv)
CNN=CNN.to(device)
print(CNN)

In [ ]:
test_input = torch.Tensor(3,3,56,56).to(device)

out = CNN(test_input)
print(out.shape)

In [ ]:
optimizer = optim.SGD(CNN.parameters(),lr=0.001,momentum=0.9)
loss_function = nn.CrossEntropyLoss()

lr_sche = optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma=0.9)

In [ ]:
loss_plot = vis.line(Y=torch.Tensor(1).zero_())

In [ ]:
epochs = 50
print("length of train_loader : {}".format(len(train_loader)))
for epoch in range(epochs):
    running_loss = 0.0
    check_loss = 0.0
    lr_sche.step()
    for num, data in enumerate(train_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = CNN(inputs)
        loss = loss_function(out, labels)
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()
        check_loss += loss.item()
        
        if (num % 30 == 1) and (num != 1):
            check_loss = check_loss/30
            print("loss : {}".format(check_loss))
            plt_tracker(loss_plot,(check_loss), epoch *len(train_loader) + num)
            check_loss = 0  
        
        if num % len(train_loader) == (len(train_loader) - 1):
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, num + 1, running_loss / len(train_loader)))
            running_loss = 0.0
            check_loss=0.0

In [ ]:
torch.save(CNN.state_dict(),"./model/tiny_imagenet_vgg.pth")

In [ ]:
load_conv = vgg.make_layers(vgg.cfg['tiny-imagenet'],True)
load_CNN = tiny_VGG(load_conv)

In [ ]:
load_CNN.load_state_dict(torch.load('./model/tiny_imagenet_vgg.pth',map_location = device))
load_CNN = load_CNN.to(device)

In [ ]:
print(load_CNN.conv[0].weight.data[0][0])
print(load_CNN.conv[0].bias.data[0])

In [ ]:
print(CNN.conv[0].weight.data[0][0])
print(CNN.conv[0].bias.data[0])